# Client

Demo of a client interacting with a remote agent that can use history.

You can interact with this via API directly

In [1]:
import requests

inputs = {"input": {"input": "what is the length of the word audioee?", "chat_history": []}}
response = requests.post("http://localhost:8000/invoke", json=inputs)

response.json()

{'output': {'output': 'The length of the word "audioee" is 7.'},
 'callback_events': [],
 'metadata': {'run_id': '468ae194-09e3-4318-b13f-724b283b8c4d'}}

You can also interact with this via the RemoteRunnable interface (to use in other chains)

In [2]:
from langserve import RemoteRunnable

remote_runnable = RemoteRunnable("http://localhost:8000/")

Remote runnable has the same interface as local runnables

In [3]:
from langchain_core.messages import HumanMessage, AIMessage

In [4]:
chat_history = []

while True:
    human = input("Human (Q/q to quit): ")
    if human in {"q", "Q"}:
        print('AI: Bye bye human')
        break
    ai = await remote_runnable.ainvoke({"input": human, "chat_history": chat_history})
    print(f"AI: {ai['output']}")
    chat_history.extend([HumanMessage(content=human), AIMessage(content=ai['output'])])

Human (Q/q to quit):  hello


AI: Hello! How can I assist you today?


Human (Q/q to quit):  my name is eugene


AI: Nice to meet you, Eugene! How can I help you today?


Human (Q/q to quit):  what's your name


AI: I am an AI assistant and I don't have a personal name. You can simply call me "Assistant". How can I assist you further?


Human (Q/q to quit):  what's was my name and what's its length?


AI: Your name is Eugene and it has a length of 6 characters. Is there anything else you would like to know?


Human (Q/q to quit):  q


AI: Bye bye human


## Stream

Please note that streaming alternates between actions and observations. It does not stream individual tokens!

Please reference LangChain documentation to see how to stream individual tokens. 

At the time of writing 2024-01-18, the way to do this is using *astream_log*, and there's a proposed API using *astream_events* that will be merged soon.

In [6]:
chat_history = []

while True:
    human = input("Human (Q/q to quit): ")
    if human in {"q", "Q"}:
        print('AI: Bye bye human')
        break

    ai = None
    print("AI: ")
    async for chunk in remote_runnable.astream({"input": human, "chat_history": chat_history}):
        # Agent Action
        if "actions" in chunk:
            for action in chunk["actions"]:
                print(
                    f"Calling Tool ```{action['tool']}``` with input ```{action['tool_input']}```"
                )
        # Observation
        elif "steps" in chunk:
            for step in chunk["steps"]:
                print(f"Got result: ```{step['observation']}```")
        # Final result
        elif "output" in chunk:
            print(chunk['output'])
            ai = AIMessage(content=chunk['output'])
        else:
            raise ValueError
        print("------")        
    chat_history.extend([HumanMessage(content=human), ai])

Human (Q/q to quit):  hello


AI: 
Hello! How can I assist you today?
------


Human (Q/q to quit):  my name is eugene


AI: 
Nice to meet you, Eugene! How can I help you today?
------


Human (Q/q to quit):  what's the length of my name?


AI: 
Calling Tool ```word_length``` with input ```{'word': 'eugene'}```
------
Got result: ```6```
------
The length of your name, Eugene, is 6 characters.
------


Human (Q/q to quit):  q


AI: Bye bye human
